In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## importing required libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import RandomOverSampler

In [ ]:
data = pd.read_csv('../input/loan-prediction-based-on-customer-behavior/Training Data.csv')

In [ ]:
data.head()

In [ ]:
sns.heatmap(data.corr(), annot=True)

In [ ]:
data.dtypes

In [ ]:
data['Married/Single'].unique()

In [ ]:
data['House_Ownership'].unique()

In [ ]:
data['Car_Ownership'].unique()

So we can apply one hot encoding in Married/Single, House_Ownership, Car_Ownership column

Also id, city, profession and state column is not needed, so we can drop that

In [ ]:
df = data.copy()

In [ ]:
df.drop(columns=['Id', 'STATE', 'CITY', 'Profession'], inplace=True, axis=1)

In [ ]:
def enc(data1):
    df = pd.get_dummies(data = data1, columns=['Married/Single', 'House_Ownership', 'Car_Ownership'])
    
    return df

In [ ]:
df = enc(df)

In [ ]:
df

Dividing Data into X and Y

In [ ]:
X = df.drop(columns=['Risk_Flag'])
y = df['Risk_Flag']

In [ ]:
y.value_counts()

Here we can see that our target variable has much more number of 0's than 1's. So this will make our model bias and reduce its efficiency. So we will OverSample it to make it equal and reduce Bias.

In [ ]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X,y)
y_ros.value_counts()

Now we can see that both values are equal. And our bias is also reduced.
Now we will split our data into 80% train set and 20% test/val data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ros,y_ros, test_size = 0.2, random_state=15)

Trying Randomized search for best XGB Parameters. You can also try Grid Search but that consumes more time in CPU.

Also commenting out randomized search, you can try it at your own.

You can try different values here and see which works best for your model.

In [ ]:
#params = {
#    'learning_rate' : [0.1,0.3,0.5],
#    'n_estimators' : [700, 900, 1000],
#    'max_depth' : [4, 6, 9]
#}

# xgb = XGBClassifier()

# skf = StratifiedKFold(n_splits=5 , shuffle=True, random_state=2021)

# random_search  =RandomizedSearchCV(xgb, param_distributions=params , n_iter=9, scoring='roc_auc' , n_jobs=4, cv=skf.split(X_train,y_train), random_state=2020)

# random_search.fit(X_train,y_train)

#print(random_search.best_params_)

In [ ]:
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
predict_train = xgb.predict(X_train)
predict_test = xgb.predict(X_test)

In [ ]:
print("Train Score:", roc_auc_score(predict_train, y_train))
print("Train error:", mean_squared_error(predict_train,y_train))
print("Test Score:", roc_auc_score(predict_test, y_test))
print("Test error:", mean_squared_error(predict_test,y_test))

Plotting our predicted values

In [ ]:
def plot(y_true, pred, title=None):
    
    matrix = confusion_matrix(y_true, pred)
    plt.figure(figsize = (5,5))
    sns.heatmap(data= matrix, annot=True, cmap='Blues', fmt='g')
    
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.title(title)
    plt.show

In [ ]:
plot(y_train,predict_train, "Training Predictions")

In [ ]:
plot(y_test,predict_test, "Test Predictions")